<a href="https://colab.research.google.com/github/kyanite153/Prog2kakushin/blob/main/work2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 初めに実行

In [23]:
import numpy as np
import random

class Sudoku:
  def __init__(self, sudoku=None):
    self.sudoku = np.zeros([9,9], int)
    self.answer = None
    if type(sudoku) != np.ndarray:
      self.create()
    else:
      self.sudoku = sudoku[0]
      self.answer = sudoku[1]
  def create(self):
    while True:
      self.sudoku = np.zeros([9,9], int)
      self.sudoku[0] = np.array(random.sample(range(1,10),9))
      for i in range(len(self.sudoku)*(len(self.sudoku)-1)):
        a = random.sample(range(1,10),9)
        for j in range(9):
          self.sudoku[i//len(self.sudoku)+1][i%len(self.sudoku)] = a[j]
          if self.is_integrity() == True:
            break
      if self.is_integrity(perfect=True) == True:
        break
    self.answer = self.sudoku
    count = 0
    while True:
      l = []
      for i in range(81):
        if self.sudoku[i//len(self.sudoku)][i%len(self.sudoku)] != 0:
          l.append(i)
      l = random.sample(l, len(l))
      a = random.choice(l)
      new_sudoku = self.sudoku.copy()
      new_sudoku[a//len(self.sudoku)][a%len(self.sudoku)] = 0
      if True:
        #理想は「答えがただ一つ決まるとき」を条件にすることだが、代わりに空白の数を指定している．
        self.sudoku = new_sudoku
        count += 1
        if count >= 40:
          break
  @staticmethod
  def is_correct(sudoku, perfect=False):
    if perfect == True:
      if np.where(sudoku==0,1,0).sum() != 0:
        return False
    #横
    for row in sudoku:
      zero = np.where(row==0,1,0).sum()
      if len(set(row)-{0}) != len(row)-zero:
        return False
    #縦
    for col in sudoku.T:
      zero = np.where(col==0,1,0).sum()
      if len(set(col)-{0}) != len(col)-zero:
        return False
    #ブロック
    a = np.array_split(sudoku, 3, 0)
    blocks = []
    for x in a:
      [blocks.append(y) for y in (np.array_split(x, 3, 1))]
    for block in blocks:
      zero = np.where(block.flatten()==0,1,0).sum()
      if len(set(block.flatten())-{0}) != len(block.flatten())-zero:
        return False
    return True
  @staticmethod
  def solve(sudoku, i=0):
    sudoku = np.copy(sudoku)
    if i >= 81:
      #すべての値が埋まっている(と予想される)とき
      if Sudoku.is_correct(sudoku, perfect=True) == True:
        return sudoku
      else:
        return False
    #←ここに、値がただ一つに決まるかを調べるプログラムを書く．(書かない場合処理に非常に時間がかかる．)
    if sudoku[i//len(sudoku)][i%len(sudoku)] != 0:
      x = Sudoku.solve(sudoku, i=i+1)
      if type(x) == np.ndarray:
        return x
    a = random.sample(range(1,10),9)
    for j in range(9):
      sudoku[i//len(sudoku)][i%len(sudoku)] = a[j]
      if Sudoku.is_correct(sudoku) == False:
        continue
      x = Sudoku.solve(sudoku, i=i+1)
      if type(x) == np.ndarray:
        return x
    return False
  def is_integrity(self, perfect=False):
    return self.is_correct(self.sudoku, perfect)
  def show(self, mode='q'):
    if mode=='q':
      print(self.sudoku)
    elif mode=='a':
      print(self.answer)

def main():
  sudoku_list = []
  while True:
    print('1:問題の自動生成')
    print('2:問題の自動解析')
    print('3:これまでに作成/入力した問題のリストを表示')
    print('4:プログラムを終了')
    ans = int(input('ここに数字で入力：'))
    print()
    if ans == 1:
      print('問題の自動生成を行います')
      print('問題生成中...')
      sudoku = Sudoku()
      print('生成完了')
      sudoku.show()
      print('1:答えを表示')
      print('2:メインメニューに戻る')
      ans2 = int(input('ここに数字で入力：'))
      if ans2 == 1:
        print(' 答え')
        sudoku.show(mode='a')
      sudoku_list.append(sudoku)
    elif ans == 2:
      print('自動解析する問題を以下から選ぶ')
      print('1:問題リストから選ぶ')
      print('2:他の問題を入力')
      print('3:メインメニューに戻る')
      ans2 = int(input('ここに数字で入力：'))
      sudoku = None
      if ans2 == 1:
        if len(sudoku_list) == 0:
          print('リストが空です．')
        else:
          ans3 = int(input(f'1～{len(sudoku_list)}の数字を入力：'))
          sudoku = sudoku_list[ans3-1].sudoku
      elif ans2 == 2:
        sudoku = np.zeros([9,9], int)
        print('上から列ごとの数字を入力(例：「153798264」)')
        for i in range(9):
          row = input(f'{i+1}列目：')
          sudoku[i] = np.array(list(map(int,row)))
        print(sudoku)
      if type(sudoku) == np.ndarray:
        print('解析開始')
        print('解析中...')
        sudoku_ans = Sudoku.solve(sudoku)
        if type(sudoku) == np.ndarray:
          print('解析完了')
          print(sudoku_ans)
          if ans2 == 2:
            sudoku_list.append(Sudoku(sudoku=np.array([sudoku,sudoku_ans])))
        else:
          print('解析不能：問題に不備がある可能性あり')
    elif ans == 3:
      if len(sudoku_list) == 0:
        print('リストが空です．')
      else:
        ans3 = int(input(f'1～{len(sudoku_list)}の数字を入力：'))
        sudoku = sudoku_list[ans3-1]
        sudoku.show()
        print('1:答えを表示')
        print('2:メインメニューに戻る')
        ans2 = int(input('ここに数字で入力：'))
        if ans2 == 1:
          print(' 答え')
          sudoku.show(mode='a')
    elif ans == 4:
      #終了
      break
    print()


# メイン

In [ ]:
if __name__ == '__main__':
  main()